In [1]:
#imports
import cv2 
import numpy as np
from object_detection import ObjectDetection
import math

In [ ]:
#Init Object Detection 
od = ObjectDetection()

In [ ]:
#Start video capture
#Replace "los_angeles.mp4" with any traffic footage
vidCap = cv2.VideoCapture("los_angeles.mp4")

frameCount = 0
center_pts_prev_frame = []
tracking_objs = {}
trackId = 0

while True: 
    moreFrames, frame = vidCap.read()
    frameCount += 1
    if not moreFrames: 
        break
        
    center_pts_cur_frame = []
        
    # Detect objs on each frame 
    (class_id, scores, boxes) = od.detect(frame)
    for box in boxes: 
        (x, y, w, h) = box
        center_x = int((x+x+w)/2)
        center_y = int((y+y+h)/2)
        center_pts_cur_frame.append((center_x, center_y))
        print("Frame #",frameCount,"Coords: ",x,y,w,h) #prints [x, y, width, height]
        
        cv2.rectangle(frame, (x, y),(x+w, y+h), (255, 0, 255), 2) #top-left and bottom-right points to draw rectangle
    if frameCount <= 2:
        print('too early')
    else:
        center_pts_cur_frame_copy = center_pts_cur_frame.copy()
        
        for object_id, pt2 in tracking_objs.copy().items():
            onScreen = False
            
            for pt in center_pts_cur_frame_copy:
                dist = math.hypot(pt2[0]-pt[0],pt2[1]-pt[1])
                #Update obj position 
                if dist < 20:
                    tracking_objs[object_id] = pt
                    onScreen = True
                    #After position has been assigned do not compare to other objs
                    if pt in center_pts_cur_frame: 
                        center_pts_cur_frame.remove(pt)
                        continue 
            #Remove ID bc car not on screen 
            if not onScreen: 
                tracking_objs.pop(object_id)
        #Add new IDs as cars are found/initialize IDs
        for pt in center_pts_cur_frame:
            tracking_objs[trackId] = pt
            trackId += 1
        
            
    
    for object_id, pt in tracking_objs.items(): 
        cv2.circle(frame, pt, 3, (0,0,255), -1)
        cv2.putText(frame, str(object_id), (pt[0],pt[1]-10),0,1,(0,0,255),2)
        
        
    print('Tracking obs', tracking_objs)
    print("CUR FRAME")
    print(center_pts_cur_frame)
    cv2.imshow("Frame", frame)
    
    #Making copy of points
    center_pts_prev_frame = center_pts_cur_frame.copy()
    
    key = cv2.waitKey(30)
    if key == 27: #'s' key
        break
cap.release()
cap.destroyAllWindows()
        